# Solving a complex task with a multi-agent hierarchy

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

> 👉 Find all Batman filming locations in the world, calculate the time to transfer via boat to there, and represent them on a map, with a color varying by boat transfer time. Also represent some supercar factories with the same boat transfer time.

Let's build this!

In [1]:
!pip install 'smolagents[litellm]' matplotlib geopandas shapely kaleido -q

# smolagents with the litellm extra

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates iterating over tupples components
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [4]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, HfApiModel, VisitWebpageTool


# model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together") # together not yet available

model=HfApiModel('https://alsa3cvpzlih451k.us-east-1.aws.endpoints.huggingface.cloud')

We can start with creating a baseline, simple agent to give us a simple report.

In [5]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [6]:
from google.colab import userdata
import os
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

In [7]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(provider="serpapi"), VisitWebpageTool(), calculate_cargo_travel_time], # GoogleSearchTool(provider="serper")!!!!
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [8]:
# Don't forget to run the endpoint before running this cell:
# https://endpoints.huggingface.co/Clod62/endpoints/qwen2-5-coder-32b-instruct-g-fwz

result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - https://alsa3cvpzlih451k.us-east-1.aws.endpoints.huggingface.cloud ───────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Search for Batman filming locations                                                                            
  batman_locations = web_search(query="Batman filming locations worldwide")                                        
  print("Batman filming locations:", batman_locations)                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Batman filming locations: ## Search Results
0. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

1. [Category:Film Locations | Batman Wiki - Fandom](https://batman.fandom.com/wiki/Category:Film_Locations)
Source: Batman Wiki

Film Locations · A. Arctic World · B Batcave (Burtonverse) · C Cave of Horrors · D Category:Dark Knight Trilogy 
locations · F.

2. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

3. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery) Helpful•86 1
St. George's Hall, Liverpool, England, UK

4. [Dark Knight Rises Tour: See Batman Movie 
Locations](https://www.travelchannel.com/interests/arts-and-culture/photos/see-batman-movie-locations)
Source: Travel Channel

See Batman Movie Locations · Wollaton Hall · Carnegie Mellon University · The Farmiloe Building · Queensboro Bridge
· Wall Street · Mehrangarh Fort · Turda Saline.

5. [What cities in America and anywhere in the world do you 
...](https://www.reddit.com/r/batman/comments/1d1t88q/what_cities_in_america_and_anywhere_in_the_world/)
Source: Reddit · r/batman

Glasgow and Liverpool were used in the shoot for The Batman. I'd avoid Chicago, as that looks more like Metropolis 
than Gotham.

6. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

The Batman was primarily filmed in the United Kingdom. Most of the recognizable buildings of this new version of 
Gotham City are located in Liverpool.

7. [The Batman movie locations and other Wayne Manors you 
...](https://www.lifestyleasia.com/sg/entertainment/the-batman-movie-locations-around-the-world-you-can-visit/)
Date published: Mar 7, 2022
Source: Lifestyle Asia Hong Kong

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh.

8. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations. Knebworth House, Knebworth, Hertfordshire, England, UK ... Marvel or DC? Christian Bale in The 
Dark Knight (2008). Worldwide Box Office.

9. [In Time / Batman location? : 
r/whereisthis](https://www.reddit.com/r/whereisthis/comments/146xczi/in_time_batman_location/)
Source: Reddit · r/whereisthis

It runs under 6th Street Bridge, and leads down to the LA Riverbed. This was shot on the west side of the LA River 
just south of the corner of Mesquit Street & ...

Out: None

[Step 1: Duration 21.97 seconds| Input tokens: 2,399 | Output tokens: 113]

...

[Step 4: Duration 17.57 seconds| Input tokens: 14,785 | Output tokens: 1,841]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Extract supercar factories and their coordinates                                                               
  supercar_factories_dict = {                                                                                      
      "Bugatti, Molsheim, France": (48.6702, 7.6813),                                                              
      "Ferrari, Maranello, Italy": (43.8175, 10.9836),                                                             
      "Lamborghini, Sant'Agata Bolognese, Italy": (44.4859, 11.3753),                                              
      "Pagani, San Cesario sul Panaro, Italy": (44.7682, 10.3667),                                                 
      "Gordon Murray Automotive, Gordon, UK": (51.0975, 0.1845),                                                   
      "Koenigsegg, Angelholm, Sweden": (56.2745, 12.8977),                                                         
  }                                                                                                                
                                                                                                                   
  # Gotham coordinates                                                                                             
  gotham_coords = (40.7128, -74.0060)                                                                              
                                                                                                                   
  # Calculate travel time for each supercar factory                                                                
  supercar_travel_times = []                                                                                       
  for location, coords in supercar_factories_dict.items():                                                         
      travel_time = calculate_cargo_travel_time(origin_coords=coords, destination_coords=gotham_coords)            
      supercar_travel_times.append({"Location": location, "Travel Time (hours)": travel_time})                     
                                                                                                                   
  # Create DataFrame with supercar factories and their travel times                                                
  supercar_df = pd.DataFrame(supercar_travel_times)                                                                
                                                                                                                   
  # Concatenate Batman and supercar DataFrames                                                                     
  combined_df = pd.concat([batman_df, supercar_df], ignore_index=True)                                             
                                                                                                                   
  # Print combined DataFrame                                                                                       
  print(combined_df)                                                                                               
                                                                                                                   
  # Return final answer                                                                                            
  final_answer(combined_df)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
                                            Location  Travel Time (hours)
0         Necropolis Cemetery, Glasgow, Scotland, UK                 8.60
1          St. George's Hall, Liverpool, England, UK                 8.81
2  Knebworth House, Knebworth, Hertfordshire, Eng...                 9.16
3                          Bugatti, Molsheim, France                10.10
4                          Ferrari, Maranello, Italy                10.79
5           Lamborghini, Sant'Agata Bolognese, Italy                10.78
6              Pagani, San Cesario sul Panaro, Italy                10.65
7               Gordon Murray Automotive, Gordon, UK                 9.22
8                      Koenigsegg, Angelholm, Sweden                10.06

Out - Final answer:                                             Location  Travel Time (hours)
0         Necropolis Cemetery, Glasgow, Scotland, UK                 8.60
1          St. George's Hall, Liverpool, England, UK                 8.81
2  Knebworth House, Knebworth, Hertfordshire, Eng...                 9.16
3                          Bugatti, Molsheim, France                10.10
4                          Ferrari, Maranello, Italy                10.79
5           Lamborghini, Sant'Agata Bolognese, Italy                10.78
6              Pagani, San Cesario sul Panaro, Italy                10.65
7               Gordon Murray Automotive, Gordon, UK                 9.22
8                      Koenigsegg, Angelholm, Sweden                10.06

[Step 5: Duration 87.10 seconds| Input tokens: 21,375 | Output tokens: 2,708]

In [9]:
result

,Location,Travel Time (hours)
0,"Necropolis Cemetery, Glasgow, Scotland, UK",8.60
1,"St. George's Hall, Liverpool, England, UK",8.81
2,"Knebworth House, Knebworth, Hertfordshire, Eng...",9.16
3,"Bugatti, Molsheim, France",10.10
4,"Ferrari, Maranello, Italy",10.79
5,"Lamborghini, Sant'Agata Bolognese, Italy",10.78
6,"Pagani, San Cesario sul Panaro, Italy",10.65
7,"Gordon Murray Automotive, Gordon, UK",9.22
8,"Koenigsegg, Angelholm, Sweden",10.06


We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [ ]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - https://alsa3cvpzlih451k.us-east-1.aws.endpoints.huggingface.cloud ───────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
```
1. Use `web_search` to find a list of Batman filming locations worldwide from sources like IMDb, TheWrap, and The 
New York Times.
2. For each filming location identified, use `visit_webpage` to confirm the information and gather the exact 
address or coordinates.
3. Convert the addresses of Batman filming locations to coordinates using `visit_webpage` to search Google Maps.
4. Use `calculate_cargo_travel_time` to calculate the travel time from each Batman filming location to Gotham using
the coordinates obtained.
5. Use `web_search` to find a list of supercar factories worldwide from automotive websites or company official 
sites.
6. For each supercar factory identified, use `visit_webpage` to confirm the information and gather the exact 
address or coordinates.
7. Convert the addresses of supercar factories to coordinates using `visit_webpage` to search Google Maps.
8. Use `calculate_cargo_travel_time` to calculate the travel time from each supercar factory to Gotham using the 
coordinates obtained.
9. Create a pandas dataframe with the Batman filming locations and their corresponding cargo plane transfer times 
to Gotham.
10. Filter the supercar factories that have the same cargo plane transfer time to Gotham as the Batman filming 
locations.
11. Add the filtered supercar factories and their transfer times to the pandas dataframe.
12. Use `final_answer` to return the pandas dataframe containing Batman filming locations and matching supercar 
factories with the same cargo plane transfer time to Gotham.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_filming_locations = web_search(query="Batman filming locations worldwide")                                
  print(batman_filming_locations)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

1. [Category:Film Locations | Batman Wiki - Fandom](https://batman.fandom.com/wiki/Category:Film_Locations)
Source: Batman Wiki

Film Locations · A. Arctic World · B Batcave (Burtonverse) · C Cave of Horrors · D Category:Dark Knight Trilogy 
locations · F.

2. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

3. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery) Helpful•86 1
St. George's Hall, Liverpool, England, UK

4. [Dark Knight Rises Tour: See Batman Movie 
Locations](https://www.travelchannel.com/interests/arts-and-culture/photos/see-batman-movie-locations)
Source: Travel Channel

See Batman Movie Locations · Wollaton Hall · Carnegie Mellon University · The Farmiloe Building · Queensboro Bridge
· Wall Street · Mehrangarh Fort · Turda Saline.

5. [What cities in America and anywhere in the world do you 
...](https://www.reddit.com/r/batman/comments/1d1t88q/what_cities_in_america_and_anywhere_in_the_world/)
Source: Reddit · r/batman

Glasgow and Liverpool were used in the shoot for The Batman. I'd avoid Chicago, as that looks more like Metropolis 
than Gotham.

6. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

The Batman was primarily filmed in the United Kingdom. Most of the recognizable buildings of this new version of 
Gotham City are located in Liverpool.

7. [The Batman movie locations and other Wayne Manors you 
...](https://www.lifestyleasia.com/sg/entertainment/the-batman-movie-locations-around-the-world-you-can-visit/)
Date published: Mar 7, 2022
Source: Lifestyle Asia Hong Kong

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh.

8. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations. Knebworth House, Knebworth, Hertfordshire, England, UK ... Marvel or DC? Christian Bale in The 
Dark Knight (2008). Worldwide Box Office.

9. [In Time / Batman location? : 
r/whereisthis](https://www.reddit.com/r/whereisthis/comments/146xczi/in_time_batman_location/)
Source: Reddit · r/whereisthis

It runs under 6th Street Bridge, and leads down to the LA Riverbed. This was shot on the west side of the LA River 
just south of the corner of Mesquit Street & ...

Out: None

...

Error in generating model output:
(ReadTimeoutError("HTTPSConnectionPool(host='alsa3cvpzlih451k.us-east-1.aws.endpoints.huggingface.cloud', 
port=443): Read timed out. (read timeout=120)"), '(Request ID: ebeb682a-6b0c-4fee-983c-88fa5677cf4a)')

[Step 7: Duration 120.24 seconds| Input tokens: 42,410 | Output tokens: 2,162]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
(ReadTimeoutError("HTTPSConnectionPool(host='alsa3cvpzlih451k.us-east-1.aws.endpoints.huggingface.cloud', 
port=443): Read timed out. (read timeout=120)"), '(Request ID: 70d34e87-1f45-4a09-b6ef-bf3ae9f569b3)')

[Step 8: Duration 121.25 seconds| Input tokens: 49,358 | Output tokens: 2,612]

In [ ]:
detailed_report

Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `matplotlib`, and `geopandas` + `shapely` for spatial plotting.

In [ ]:
model = HfApiModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
)

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serper"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [ ]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=HfApiModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [ ]:
manager_agent.visualize()

In [ ]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

I don't know how that went in your run, but in mine, the manager agent skilfully divided tasks given to the web agent in `1. Search for Batman filming locations`, then `2. Find supercar factories`, before aggregating the lists and plotting the map.

Let's see what the map looks like by inspecting it directly from the agent state:

In [ ]:
manager_agent.python_executor.state["fig"]

![output map](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/smolagents/output_map.png)